<a href="https://colab.research.google.com/github/fizzious1/WikiLinksExplore/blob/main/Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia-api networkx matplotlib

In [568]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [569]:
!git clone https://github.com/fizzious1/WikiLinksExplore

Cloning into 'WikiLinksExplore'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [570]:
%cd WikiLinksExplore


/content/WikiLinksExplore


In [573]:
  git config user.email "arieleingal121@gmail.com"
  git config user.name "fizzious1"

SyntaxError: invalid syntax (<ipython-input-573-f8de1d984f6f>, line 1)

In [574]:
!git add .
!git commit -m "Initial commit from Google Colab"


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@28bc5adbc28b.(none)')


In [551]:
import wikipediaapi
import networkx as nx
import matplotlib.pyplot as plt
from sortedcontainers import SortedList, SortedSet, SortedDict
import time

user_agent = 'WikiLinkExplorer/1.0 (mailto:arieleingal121@gmail.com) Python/3.10'
wiki_wiki = wikipediaapi.Wikipedia(user_agent)

In [567]:
garbage_links_table = []
paths_table = SortedDict()
first_six_of_page_table = SortedDict()
big_six_table = SortedDict()
succesful_paths = 0
failed_paths = 0

In [416]:
import requests
from bs4 import BeautifulSoup

def get_first_six_links(page_name):
    url = f"https://en.wikipedia.org/w/api.php"
    params = {
        "action": "parse",
        "page": page_name,
        "prop": "text",
        "format": "json",
        "section": 0
    }
    response = requests.get(url, params=params)
    data = response.json()

    if "error" in data:
        print(f"Error retrieving page: {data['error']['info']}")
        return []

    html_content = data["parse"]["text"]["*"]
    soup = BeautifulSoup(html_content, "html.parser")

    non_italic_links = []

    # Find the first six non-italic, non-parentheses, unique links in the content
    for p_tag in soup.find_all("p"):
        text = p_tag.get_text()  # Get the text of the paragraph
        inside_parentheses = False

        for a_tag in p_tag.find_all("a", href=True):
            # Track whether we're inside parentheses
            link_position = text.find(a_tag.get_text())
            before_link_text = text[:link_position]
            if '(' in before_link_text and ')' not in before_link_text:
                inside_parentheses = True
            if ')' in before_link_text:
                inside_parentheses = False

            # Check if the link meets all criteria
            href = a_tag["href"]
            if (
                a_tag.find_parent(["i", "em"]) is None and  # Not italic
                not inside_parentheses and  # Not in parentheses
                href.startswith("/wiki/") and  # Valid Wikipedia link
                not href.startswith("/wiki/Special:") and  # Not a special page
                "#" not in href and  # Exclude links that lead to a section within the same page
                ':' not in href  # Exclude links with a colon (e.g., Category:, File:)
            ):
                link_name = href.split("/wiki/")[1].replace('_', ' ')
                non_italic_links.append(link_name)

                # Stop once we have six links
                if len(non_italic_links) >= 6:
                    return non_italic_links

    return non_italic_links  # Return the list, even if it has fewer than 6 links

# Example usage
first_six_non_italic_non_parentheses_unique_links = get_first_six_links("Computer science")
print(f"The first six non-italic, non-parentheses, unique links on the 'Computer science' page are: {first_six_non_italic_non_parentheses_unique_links}")


The first six non-italic, non-parentheses, unique links on the 'Computer science' page are: ['Computation', 'Information', 'Automation', 'Theoretical computer science', 'Applied science', 'Computer architecture']


In [458]:
def save_first_six_of_page(page_name):
  first_six_of_page_table[page_name] = get_first_six_links(page_name)[:6]

In [371]:
def update_big_six_table(page):
    page_name = page.title
    if first_six_of_page_table.__contains__(page_name) == False:
      #first_six_of_page_table[page_name] = []
      save_first_six_of_page(page_name)

    first_six = first_six_of_page_table[page_name]
    #first_six = get_first_10_links(page_name)[:6]
    counter = 0
    for link in first_six:
      #title = link.title()
      if big_six_table.__contains__(link) == False:
        big_six_table[link] = [0] * 6
      big_six_table[link][counter] += 1
      counter += 1




In [559]:
def update_paths_table(page, page_to_add):
  page_name = page.title
  if paths_table.__contains__(page_name) == False:
    paths_table[page_name] = []
  first_six = get_first_six_links(page_to_add.title)[:6]
  if first_six:
    next_page = first_six[0]
    paths_table[page_name].append(next_page)


In [560]:

def find_path(initial_page):
    global succesful_paths
    global failed_paths
    current_page = initial_page
    path = []
    while current_page.title != 'Philosophy':
        path.append(current_page.title)
        update_big_six_table(current_page)
        update_paths_table(initial_page, current_page)
        print(first_six_of_page_table)
        if current_page.title in first_six_of_page_table:
            if first_six_of_page_table[current_page.title]:
             next_page_title = first_six_of_page_table[current_page.title][0]
             current_page = wiki_wiki.page(next_page_title)
             time.sleep(1)  # Wait for 1 second
            else:
              garbage_links_table.append(path)
              print("This Path Is Dead End")
              failed_paths += 1
              break
        else:
            break
    print("Found my way to Philosophy!")
    succesful_paths += 1

----TESTS----

In [507]:
print(paths_table)
print(first_six_of_page_table)
print(big_six_table)

SortedDict({})
SortedDict({})
SortedDict({})


In [566]:
def print_paths_table(paths_table):
    print("Paths Table:")
    for key in paths_table:
        # Correctly get the title of each page
        path_string = "->".join([page.title() for page in paths_table[key]])
        print(f"{key}: {path_string}")

# Example usage with your paths_table
print_paths_table(paths_table)


Paths Table:
/pol/: Political Discussion
1776 Returns: Proud Boys->Far-Right Politics->Ultraconservatism->Conservative
1928 German federal election: 


In [556]:
print("\nGarbage Links Table:")
print(garbage_links_table)


Garbage Links Table:
[['/pol/', 'Political discussion'], ['1776 Returns', 'Proud Boys', 'Far-right politics', 'Ultraconservatism', 'Conservative'], ['1928 German federal election', 'Fourth Marx cabinet', 'Wilhelm Marx', 'Centre Party (Germany)', 'Christian democracy', 'Political ideology'], ['1929 German referendum'], ['1930 German federal election', 'First Br%C3%BCning cabinet']]


In [557]:
print("Paths Table:")
for key in paths_table:
  print(key)
  for path in paths_table[key]:
    print(path.title())
print("\nFirst Six of Page Table:")
print(first_six_of_page_table)
print("\nBig Six Table:")
print(big_six_table)
print("\nGarbage Links Table:")
print(garbage_links_table)
print("\nSuccsessful Paths:")
print(succesful_paths)
print("\Failed Paths:")
print(failed_paths)

Paths Table:
/pol/
Political Discussion
Political Discussion
1776 Returns
Proud Boys
Proud Boys
Proud Boys
Proud Boys
Proud Boys
1928 German federal election
Fourth Marx Cabinet
Fourth Marx Cabinet
Fourth Marx Cabinet
Fourth Marx Cabinet
Fourth Marx Cabinet
Fourth Marx Cabinet
1929 German referendum
1930 German federal election
First Br%C3%Bcning Cabinet
First Br%C3%Bcning Cabinet
1932 German presidential election
Paul Von Hindenburg

First Six of Page Table:
SortedDict({'/pol/': ['Political discussion', 'Imageboard', '4chan', 'Internet culture', 'Far-right extremism', 'Racism'], '1776 Returns': ['Proud Boys', 'Enrique Tarrio', 'January 6 United States Capitol attack'], '1928 German federal election': ['Fourth Marx cabinet', 'Centre Party (Germany)', 'German National People%27s Party', 'German People%27s Party', 'Bavarian People%27s Party', 'Second M%C3%BCller cabinet'], '1929 German referendum': [], '1930 German federal election': ['First Br%C3%BCning cabinet', 'Centre Party (Germany)

In [563]:
pages = []

In [562]:
print(len(pages))
print(pages)

2633
['/pol/', '1776 Returns', '1928 German federal election', '1929 German referendum', '1930 German federal election', '1932 German presidential election', '1934 German referendum', '1934 Montreux Fascist conference', '1935 Saar status referendum', '1936 Summer Olympics', '1936 Winter Olympics', '1940 Field Marshal Ceremony', '1988 Hamas charter', '1st Belorussian Front', '20 July plot', '20th Century Press Archives', '21 Paths to the Kingdom of Darkness', '25-point programme', "28 May 1926 coup d'état", '30 January 1939 Reichstag speech', '4th of August Regime', '6th Army (Wehrmacht)', '6th Bavarian Reserve Division', '8chan', '9th Army (Wehrmacht)', 'A. C. Cuza', 'A. K. Chesterton', 'ADÑ–Spanish Identity', 'A History of US', 'A Racial Program for the Twentieth Century', 'A Warning to the Hindus', 'Abba Ahimeir', 'Abd al-Qadir al-Husayni', 'Abdolhossein Teymourtash', 'Abhinav Bharat', 'Abol-Ghasem Kashani', 'Academic Karelia Society', 'Academy of Fine Arts Vienna', 'Acerbo Law', 'Ac

In [564]:
# Initialize a list to store page titles
pages = []

# Fetch the page for Adolf Hitler
page = wiki_wiki.page('Adolf Hitler')

# Get the links (titles of linked pages)
links = page.links

# Iterate over the linked page titles
for link in links:
    pages.append(link)  # 'link' is already a string, no need to call title()

# Now find paths for each linked page
for page_title in pages:
    find_path(wiki_wiki.page(page_title))


SortedDict({'/pol/': ['Political discussion', 'Imageboard', '4chan', 'Internet culture', 'Far-right extremism', 'Racism']})
SortedDict({'/pol/': ['Political discussion', 'Imageboard', '4chan', 'Internet culture', 'Far-right extremism', 'Racism'], 'Political discussion': []})
This Path Is Dead End
Found my way to Philosophy!
SortedDict({'/pol/': ['Political discussion', 'Imageboard', '4chan', 'Internet culture', 'Far-right extremism', 'Racism'], '1776 Returns': ['Proud Boys', 'Enrique Tarrio', 'January 6 United States Capitol attack'], 'Political discussion': []})
SortedDict({'/pol/': ['Political discussion', 'Imageboard', '4chan', 'Internet culture', 'Far-right extremism', 'Racism'], '1776 Returns': ['Proud Boys', 'Enrique Tarrio', 'January 6 United States Capitol attack'], 'Political discussion': [], 'Proud Boys': ['Far-right politics', 'Neo-fascism', 'Militant organization', 'Political violence', 'Federal government of the United States', 'Terrorism']})
SortedDict({'/pol/': ['Politic

KeyboardInterrupt: 

In [475]:
def print_links(page):
    page_name = page.title
    links = get_first_six_links(page_name)
    for title in links:
        print("%s"% (title))

In [565]:
# # garbage_links_table = []
# # paths_table = SortedDict()
# # first_six_of_page_table = SortedDict()
# # big_six_table = SortedDict()
# # page = wiki.page('Adolf Hitler')
# # # links = page.links
# # # links.values
# # # #print(get_first_non_italic_link('Computer science'))
# # # print(get_first_non_italic_link('Computation'))
# # # page2 = wiki.page('Computation')
# # page3 = wiki.page('iMac G3')
# # find_path(page)
print(paths_table)
# print(first_six_of_page_table)
# print(big_six_table)

SortedDict({'/pol/': ['Political discussion'], '1776 Returns': ['Proud Boys', 'Far-right politics', 'Ultraconservatism', 'Conservative'], '1928 German federal election': []})
